In [1]:
import numpy as np
import pandas as pd
import gensim
from gensim.models import Doc2Vec

C:\Users\Asus\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
plot=pd.read_csv("plot_summaries.txt",header=None,sep="\t")

In [4]:
plot.head()

,0,1
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [9]:
plot.shape

(42303, 2)

In [3]:
plot.columns=["ID","Plot"]

In [4]:
df_movie=pd.read_csv("movie.metadata.tsv",header=None,sep="\t")

In [4]:
df_movie.head()

,0,1,2,3,4,5,6,7,8
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [5]:
df_movie.drop(columns=[1,4,5,6,7,8],inplace=True)

In [6]:
df_movie.shape

(81741, 3)

In [7]:
df_movie=df_movie.loc[df_movie[0].isin(plot["ID"])]

In [14]:
df_movie.shape

(42204, 3)

In [9]:
plot=plot.loc[plot.ID.isin(df_movie[0])]

In [10]:
plot.shape

(42204, 2)

In [11]:
df_movie.rename(columns={0:"ID",2:"Title",3:"Release Year"},inplace=True)

In [7]:
df_movie.head()

,ID,Title,Release Year
0,975900,Ghosts of Mars,2001-08-24
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16
2,28463795,Brun bitter,1988
3,9363483,White Of The Eye,1987
4,261236,A Woman in Flames,1983


In [12]:
df_movie["Release Year"]=(df_movie["Release Year"].str.split("-",expand=True))[0]

In [15]:
df_movie.reset_index(inplace=True, drop=True)

In [16]:
df_movie.head()

,ID,Title,Release Year
0,975900,Ghosts of Mars,2001
1,9363483,White Of The Eye,1987
2,261236,A Woman in Flames,1983
3,18998739,The Sorcerer's Apprentice,2002
4,6631279,Little city,1997


In [17]:
df=pd.merge(df_movie,plot,on="ID")

In [26]:
df.head()

,ID,Title,Release Year,Plot
0,975900,Ghosts of Mars,2001,"Set in the second half of the 22nd century, th..."
1,9363483,White Of The Eye,1987,A series of murders of rich young women throug...
2,261236,A Woman in Flames,1983,"Eva, an upper class housewife, becomes frustra..."
3,18998739,The Sorcerer's Apprentice,2002,"Every hundred years, the evil Morgana returns..."
4,6631279,Little city,1997,"Adam, a San Francisco-based artist who works a..."


In [18]:
df["Release Year"]=df["Release Year"].map(lambda x: int(x), na_action="ignore")

In [19]:
df_2=pd.read_csv("wiki_movie_plots_deduped.csv")

In [20]:
df_2.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [21]:
df_2.drop(columns=["Director", "Cast", "Wiki Page","Genre","Origin/Ethnicity"],inplace=True)

In [22]:
df_2=df_2.loc[~df_2["Title"].isin(df["Title"])]

In [23]:
df_2.shape

(14414, 3)

In [24]:
df.drop(columns=["ID"],inplace=True)

In [25]:
cols=df_2.columns.tolist()
cols[1],cols[0]=cols[0],cols[1]
cols

['Title', 'Release Year', 'Plot']

In [26]:
df_2=df_2[cols]

In [27]:
df=pd.concat([df,df_2],ignore_index=True)

In [28]:
df.shape

(56618, 3)

In [43]:
(df["Release Year"]<1960).sum()

12306

In [31]:
df=df.loc[df["Release Year"]>=1960]

In [33]:
df.sort_values(by="Release Year", inplace=True)

In [35]:
df.head()

,Title,Release Year,Plot
30589,The Magnificent Seven,1960.0,A Mexican village is periodically raided for f...
3705,College Confidential,1960.0,A sociology professor Steve McInter starts con...
9586,G.I. Blues,1960.0,U.S. Army Specialist 5 Tulsa McLean is a tan...
9567,Hell Is a City,1960.0,Committed but seen-it-all police inspector Mar...
19687,This Rebel Breed,1960.0,The film tells the tale of two policemen who g...


In [38]:
ml_movies=pd.read_csv("movies.csv")

In [39]:
ml_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [150]:
ml_movies.shape

(62423, 3)

Titles of the movies in the wikipedia plots dataset usually don't match with their correspending title in the movielens dataset. The ones in the wikipedia plots dataset have different formats so I adjust the titles in movielens dataset to 4 different formats to match the mutual movies in the datasets with minimum lost.    

In [37]:
df.Title=df.Title.map(lambda x: x[:-1] if x.endswith(" ") else x)
df.Title=df.Title.map(lambda x: x[1:] if x.startswith(" ") else x)

In [40]:
t=np.array(df.loc[df.Title.isin(ml_movies.title)].Title)

In [41]:
t.shape

(42,)

In [42]:
df["ID"] = np.nan

In [43]:
for name in t:
    df.loc[(df.Title==name)&(df.ID.isnull()),["ID"]]=ml_movies.loc[ml_movies.title==name].movieId.values[0]

In [44]:
ml_movies2 = ml_movies.drop(ml_movies.loc[ml_movies.title.isin(t)].index)

In [46]:
ml_movies2.title=ml_movies2.title.map(lambda x: x[:-7])

In [47]:
t=np.array(df.loc[df.Title.isin(ml_movies2.title)].Title)

In [48]:
t.shape

(16559,)

In [49]:
for name in t:
    df.loc[(df.Title==name)&(df.ID.isnull()),["ID"]]=ml_movies2.loc[ml_movies2.title==name].movieId.values[0]

In [50]:
df.ID.notnull().sum()

16584

In [51]:
ml_movies3=ml_movies2.drop(ml_movies2.loc[ml_movies2.title.isin(t)].index)

In [52]:
ml_movies3.shape

(44676, 3)

In [53]:
k=(ml_movies3.title.str.split("\(",expand=True))

k[0]=k[0].map(lambda x:x[:-1])

ml_movies3.title=k[0]

In [54]:
t=np.array(df.loc[df.Title.isin(ml_movies3.title)].Title)

In [55]:
t.shape

(1921,)

In [56]:
for name in t:
    df.loc[(df.Title==name)&(df.ID.isnull()),["ID"]]=ml_movies3.loc[ml_movies3.title==name].movieId.values[0]

In [57]:
df.ID.notnull().sum()

17857

In [58]:
ml_movies4=ml_movies3.drop(ml_movies3.loc[ml_movies3.title.isin(t)].index)

In [59]:
ml_movies4.shape

(42877, 3)

In [60]:
k[1]=k[1].map(lambda x:x[:-1],na_action="ignore")

ml_movies4.title=k[1]

In [61]:
t=np.array(df.loc[df.Title.isin(ml_movies4.title)].Title)

In [62]:
t.shape

(257,)

In [63]:
for name in t:
    df.loc[(df.Title==name)&(df.ID.isnull()),["ID"]]=ml_movies4.loc[ml_movies4.title==name].movieId.values[0]

In [64]:
df.ID.notnull().sum()

18058

In [65]:
df_c=df.loc[df.ID.notnull()]

In [66]:
df_c.head()

,Title,Release Year,Plot,ID
30589,The Magnificent Seven,1960.0,A Mexican village is periodically raided for f...,162350.0
9586,G.I. Blues,1960.0,U.S. Army Specialist 5 Tulsa McLean is a tan...,3602.0
9567,Hell Is a City,1960.0,Committed but seen-it-all police inspector Mar...,130422.0
13144,Spartacus,1960.0,"In the first century BC, the Roman Republic ha...",2728.0
31524,The Angel Wore Red,1960.0,A clergyman travels to Spain to join the Loyal...,124695.0


In [67]:
df_c.ID=df_c.ID.astype("int64")
df_c["Release Year"]=df_c["Release Year"].astype("int64")

C:\Users\Asus\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
ratings_df=pd.read_csv("ratings.csv")

In [69]:
ratings_df.movieId.nunique()

59047

In [70]:
ratings_df=ratings_df.loc[ratings_df.movieId.isin(df_c.ID)]

In [263]:
ratings_df.shape

(16933373, 4)

In [74]:
id_un=ratings_df.movieId.unique()

In [75]:
df_c2=df_c.loc[df_c.ID.isin(id_un)]

In [77]:
df_c2.shape

(17613, 4)

In [85]:
df_c2.to_csv("df_c.txt")

In [86]:
ratings_df.to_csv("ratings_df.txt")